<a href="https://colab.research.google.com/github/mokymok/notebooks/blob/main/discordbotwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.cloud import texttospeech

def synthesize_text(text):
    """Synthesizes speech from the input string of text."""
    client = texttospeech.TextToSpeechClient()

    input_text = texttospeech.SynthesisInput(text=text)

    # Note: the voice can also be specified by name.
    # Names of voices can be retrieved with client.list_voices().
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Standard-C",
        ssml_gender=texttospeech.SsmlVoiceGender.FEMALE,
    )

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    response = client.synthesize_speech(
        request={"input": input_text, "voice": voice, "audio_config": audio_config}
    )

    # The response's audio_content is binary.
    with open("output.mp3", "wb") as out:
        out.write(response.audio_content)
        print('Audio content written to file "output.mp3"')


In [ ]:

def explicit():
    from google.cloud import storage

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(
        'service_account.json')

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

In [ ]:
synthesize_text('hello')

DefaultCredentialsError: ignored

In [ ]:
!pip install discord

     |████████████████████████████████| 788kB 5.2MB/s 


In [ ]:
import asyncio
from configparser import ConfigParser
from os import listdir
from time import monotonic

import asyncgTTS
import asyncpg
import discord
from aiohttp import ClientSession
from discord.ext import commands

import utils

print("Starting TTS Bot!")

start_time = monotonic()
config = ConfigParser()
config.read("config.ini")
t = config["Main"]["Token"]
config_channels = config["Channels"]

cache_key_str = config["Main"]["key"][2:-1]
cache_key_bytes = cache_key_str.encode()

# Define bot and remove overwritten commands
activity = discord.Activity(name=config["Activity"]["name"], type=getattr(discord.ActivityType, config["Activity"]["type"]))
intents = discord.Intents(voice_states=True, messages=True, guilds=True, members=True)
status = getattr(discord.Status, config["Activity"]["status"])


async def prefix(bot: commands.AutoShardedBot, message: discord.Message) -> str:
    "Gets the prefix for a guild based on the passed message object"
    return await bot.settings.get(message.guild, "prefix") if message.guild else "-"


bot = commands.AutoShardedBot(
    status=status,
    intents=intents,
    help_command=None, # Replaced by FancyHelpCommand by FancyHelpCommandCog
    activity=activity,
    command_prefix=prefix,
    case_insensitive=True,
    chunk_guilds_at_startup=False,
    allowed_mentions=discord.AllowedMentions(everyone=False, roles=False)
)

bot.session = ClientSession()
bot.gtts = bot.loop.run_until_complete(
    asyncgTTS.setup(
        premium=False,
        session=bot.session
    )
)

pool = bot.loop.run_until_complete(
    asyncpg.create_pool(
        host=config["PostgreSQL Info"]["ip"],
        user=config["PostgreSQL Info"]["name"],
        database=config["PostgreSQL Info"]["db"],
        password=config["PostgreSQL Info"]["pass"]
    )
)




for cog in listdir("cogs"):
    if cog.endswith(".py"):
        bot.load_extension(f"cogs.{cog[:-3]}")
        print(f"Successfully loaded: {cog}")


@bot.event
async def on_ready():
    await bot.wait_until_ready()

    support_server_id = int(config["Main"]["main_server"])
    bot.supportserver = bot.get_guild(support_server_id)

    while bot.supportserver is None:
        print("Waiting 5 seconds")
        await asyncio.sleep(5)
        bot.supportserver = bot.get_guild(support_server_id)

    for channel_name in config_channels:
        channel_id = int(config_channels[channel_name])
        channel_object = bot.supportserver.get_channel(channel_id)
        bot.channels[channel_name] = channel_object

    try:
        await bot.starting_message.edit(content=f"~~{bot.starting_message.content}~~")
        bot.starting_message = await bot.channels["logs"].send(f"Restarted as {bot.user.name}!")
    except AttributeError:
        print(f"Logged into Discord as {bot.user.name}!")
        bot.starting_message = await bot.channels["logs"].send(f"Started and ready! Took `{monotonic() - start_time:.2f} seconds`")

print("\nLogging into Discord...")
bot.run(t)

Starting TTS Bot!


KeyError: ignored

In [ ]:
!pip install asyncgTTS

     |████████████████████████████████| 1.3MB 5.1MB/s 
     |████████████████████████████████| 296kB 18.8MB/s 
     |████████████████████████████████| 143kB 23.4MB/s 


In [ ]:
!pip install asyncpg

     |████████████████████████████████| 2.8MB 6.6MB/s 


In [ ]:
!pip install utils

In [ ]:
!pip install basic

In [ ]:
!pip install --upgrade google-cloud-texttospeech

     |████████████████████████████████| 71kB 5.7MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 


In [ ]:
!pip install --upgrade google-cloud-storage

     |████████████████████████████████| 112kB 6.8MB/s 
     |████████████████████████████████| 81kB 5.9MB/s 
ERROR: google-cloud-bigquery 1.21.0 has requirement google-resumable-media!=0.4.0,<0.5.0dev,>=0.3.1, but you'll have google-resumable-media 1.2.0 which is incompatible.
  Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
      Successfully uninstalled google-cloud-core-1.0.3
  Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Found existing installation: google-cloud-storage 1.18.1
    Uninstalling google-cloud-storage-1.18.1:
      Successfully uninstalled google-cloud-storage-1.18.1
